In [1]:
import matplotlib.pyplot as plt
from isicpy.third_party.pymatreader import hdf5todict
from isicpy.utils import makeFilterCoeffsSOS
from pathlib import Path
import h5py
import numpy as np
import pandas as pd
from scipy import signal

clinc_slines = {
    'S1_S3': 8,
    'S22': 12,
    'S18': 9,
    'S19': 7,
    'S23': 4,
    'S16': 6,
    'S15': 10,
    'S12_S20': 27,
    'S11': 17,
    'S6': 21,
    'S14': 25,
    'S7': 5,
    'S0_S2': 13
    }

clinc_sample_rate = 38690
clinc_sample_interval = pd.Timedelta(clinc_sample_rate ** -1, unit='s')
                                     
clinc_col_names = [key for key, value in clinc_slines.items()]
clinc_indexes = [value for key, value in clinc_slines.items()]
folder_path = Path(r"/users/rdarie/data/rdarie/Neural Recordings/raw/20231109-Phoenix")

file_name_list = ["MB_1699558933_985097_f.mat", "MB_1699560317_650555_f.mat", 'MB_1699560792_657674_f.mat']
# file_name = "MB_1699558933_985097_f.mat"
# file_name = "MB_1699560317_650555_f.mat"
# file_name = 'MB_1699560792_657674_f.mat'

In [ ]:
for file_name in file_name_list:
    print(f'on file {file_name}')
    file_path = folder_path / file_name
    with h5py.File(file_path, 'r') as hdf5_file:
        data = hdf5todict(hdf5_file, variable_names=['data_this_file'], ignore_fields=None)
    
    print(f"\tUnique intervals between USB packet counts: {np.unique(np.diff(data['data_this_file']['usbPacketCount'].astype(int)))}")
    print(f"\tUnique intervals between sample counts (should be just a [1] unless there are dropped packets): {np.unique(np.diff(data['data_this_file']['SampleCount'].astype(int)))}")
    
    clinc_index_temp = np.arange(
        data['data_this_file']['SampleCount'].min(),
        data['data_this_file']['SampleCount'].max() + 1, dtype=int)
    clinc_index = clinc_index_temp - data['data_this_file']['SampleCount'].min().astype(int)
    
    clinc_sync_wave = pd.Series(np.nan, index=clinc_index_temp)
    clinc_sync_wave.loc[data['data_this_file']['SampleCount']] = data['data_this_file']['SyncWave']
    clinc_sync_wave.index = clinc_index

    rising_edge_mask = ((clinc_sync_wave == 0) & (clinc_sync_wave.shift(1) == 1))
    falling_edge_mask = ((clinc_sync_wave == 1) & (clinc_sync_wave.shift(1) == 0))
    any_edge_mask = rising_edge_mask | falling_edge_mask
    
    rising_intervals = np.diff(clinc_sync_wave.index[rising_edge_mask])
    print(f'\t{rising_edge_mask.sum()} rising edges detected')
    print(f'\tUnique counts of samples between rising edges: {np.unique(rising_intervals)}')
    
    expected_period = 3
    empirical_sample_interval = expected_period / np.mean(rising_intervals)
    print(f'\tThe empirical sample rate is (# of samples between rising edges / 3 sec): {empirical_sample_interval ** -1:.2f} Hz')

on file MB_1699558933_985097_f.mat


/gpfs/home/rdarie/isi_analysis/ISI_Analysis_Python/isicpy/third_party/pymatreader/utils.py:124: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


	Unique intervals between USB packet counts: [-65535      0      1]
	Unique intervals between sample counts (should be just a [1] unless there are dropped packets): [1]
	172 rising edges detected
	Unique counts of samples between rising edges: [110795 110796]
	The empirical sample rate is (# of samples between rising edges / 3 sec): 36931.88 Hz
on file MB_1699560317_650555_f.mat


/gpfs/home/rdarie/isi_analysis/ISI_Analysis_Python/isicpy/third_party/pymatreader/utils.py:124: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


	Unique intervals between USB packet counts: [-65535      0      1]
	Unique intervals between sample counts (should be just a [1] unless there are dropped packets): [   1  260  417 1112]
	102 rising edges detected
	Unique counts of samples between rising edges: [110013 110795 110796 111578]
	The empirical sample rate is (# of samples between rising edges / 3 sec): 36931.87 Hz
on file MB_1699560792_657674_f.mat


/gpfs/home/rdarie/isi_analysis/ISI_Analysis_Python/isicpy/third_party/pymatreader/utils.py:124: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '


	Unique intervals between USB packet counts: [-65535      0      1]
	Unique intervals between sample counts (should be just a [1] unless there are dropped packets): [1]
	13 rising edges detected
	Unique counts of samples between rising edges: [110795 110796]
	The empirical sample rate is (# of samples between rising edges / 3 sec): 36931.89 Hz


In [10]:
for dsi_block_name in ['Block0001', 'Block0002']:
    dsi_df = pd.read_csv(folder_path / f"{dsi_block_name}.ascii", header=12, index_col=0, low_memory=False)
    dsi_trigs = dsi_df.iloc[:, -1]
    dsi_trigs.index = np.arange(dsi_trigs.shape[0])
    thresh = (dsi_trigs.max() - dsi_trigs.min()) / 2

    rising_edge_mask = ((dsi_trigs <= thresh) & (dsi_trigs.shift(1) >= thresh))
    falling_edge_mask = ((dsi_trigs >= thresh) & (dsi_trigs.shift(1) <= thresh))
    any_edge_mask = rising_edge_mask | falling_edge_mask
    
    rising_intervals = np.diff(dsi_trigs.index[rising_edge_mask])
    print(f'\t{rising_edge_mask.sum()} rising edges detected')
    print(f'\tUnique counts of samples between rising edges: {np.unique(rising_intervals)}')
    
    expected_period = 3
    empirical_sample_interval = expected_period / np.mean(rising_intervals)
    print(f'\tThe empirical sample rate is (# of samples between rising edges / 3 sec): {empirical_sample_interval ** -1:.2f} Hz')

	200 rising edges detected
	Unique counts of samples between rising edges: [3000 3001]
	The empirical sample rate is (# of samples between rising edges / 3 sec): 1000.00 Hz
	181 rising edges detected
	Unique counts of samples between rising edges: [3000]
	The empirical sample rate is (# of samples between rising edges / 3 sec): 1000.00 Hz
